In [1]:
pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
def initialize_driver(headers_list, proxy_list):
    options = Options()
    #select a random user-agent from the list
    user_agent = random.choice(headers_list)["User-Agent"]
    options.add_argument(f"user-agent={user_agent}")
    
    #select a random proxy from the list
    proxy = random.choice(proxy_list)
    if proxy:
        options.add_argument(f"--proxy-server={proxy}")
    
    #add some common options
    options.add_argument("--headless")
    options.add_argument("--disable-extensions")
    options.add_argument("--ignore-certificate-errors")

    #initialize Chrome WebDriver with the specified options
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    #set implicit wait of 10sec
    driver.implicitly_wait(10)

    return driver

In [5]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("--ignore-certificate-errors")

# Use ChromeDriverManager to automatically download and manage chromedriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

URL = "https://www.welcometothejungle.com/fr/jobs?page=1&refinementList%5Bprofession_name.fr.Tech%5D%5B%5D=Data%20Analysis&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI"

result = driver.get(URL)
time.sleep(10)

driver.quit()

In [18]:
import psycopg2
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("--ignore-certificate-errors")

# Use ChromeDriverManager to automatically download and manage chromedriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

URL = "https://www.welcometothejungle.com/fr/jobs?page=1&refinementList%5Bprofession_name.fr.Tech%5D%5B%5D=Data%20Analysis&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI"

result = driver.get(URL)
time.sleep(4)

# Find job elements
job_elements = driver.find_elements(By.XPATH, "//li[@data-testid='search-results-list-item-wrapper']")

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="mydatabase", 
    user="myuser", 
    password="mypassword", 
    host="localhost",  # Change to Docker container IP if running in Docker
    port="5432"
)
cursor = conn.cursor()

# Create jobs table if not exists
cursor.execute('''CREATE TABLE IF NOT EXISTS jobs (
                    id SERIAL PRIMARY KEY,
                    job_title TEXT,
                    company_name TEXT,
                    location TEXT
                )''')

# Insert job data into the database
for job_element in job_elements:
    job_title = job_element.find_element(By.XPATH, ".//h4").text
    company_name = job_element.find_element(By.XPATH, ".//span[@class='sc-ERObt ldmfCZ sc-6i2fyx-3 eijbZE wui-text']").text
    location = job_element.find_element(By.XPATH, ".//span[@class='sc-68sumg-0 gvkFZv']").text

    # Insert job data into the database
    cursor.execute("INSERT INTO jobs (job_title, company_name, location) VALUES (%s, %s, %s)", (job_title, company_name, location))

# Commit changes and close database connection
conn.commit()
conn.close()

# Close the browser
driver.quit()

print("Data stored in the database successfully.")


ModuleNotFoundError: No module named 'psycopg2'

In [19]:
pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 186.2 kB/s eta 0:00:07
   - -------------------------------------- 0.0/1.2 MB 217.9 kB/s eta 0:00:06
   --- ------------------------------------ 0.1/1.2 MB 403.5 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.2 MB 566.5 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.2 MB 538.9 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 565.6 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 680.9 kB/s eta 0:00:02
   ------------ --------------------------- 0.4/1.2 MB 794.9 kB/s eta 0:00:02
   ------------- -----

In [11]:
import time
import psycopg2
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Wait for PostgreSQL service to become available
time.sleep(10)  # Adjust the delay as needed

options = Options()
options.add_argument("--disable-extensions")
options.add_argument("--ignore-certificate-errors")

# Use ChromeDriverManager to automatically download and manage chromedriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

URL = "https://www.welcometothejungle.com/fr/jobs?page=1&refinementList%5Bprofession_name.fr.Tech%5D%5B%5D=Data%20Analysis&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI"

result = driver.get(URL)
time.sleep(4)

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname="mydatabase", 
    user="postgres", 
    password="admin", 
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Create jobs table if not exists
cursor.execute('''CREATE TABLE IF NOT EXISTS jobs (
                    id SERIAL PRIMARY KEY,
                    job_title TEXT,
                    company_name TEXT,
                    location TEXT
                )''')

# Commit changes and close database connection
conn.commit()

# Find job elements
job_elements = driver.find_elements(By.XPATH, "//li[@data-testid='search-results-list-item-wrapper']")

# Insert job data into the database
for job_element in job_elements:
    job_title = job_element.find_element(By.XPATH, ".//h4").text
    company_name = job_element.find_element(By.XPATH, ".//span[@class='sc-ERObt ldmfCZ sc-6i2fyx-3 eijbZE wui-text']").text
    location = job_element.find_element(By.XPATH, ".//span[@class='sc-68sumg-0 gvkFZv']").text

    # Insert job data into the database
    cursor.execute("INSERT INTO jobs (job_title, company_name, location) VALUES (%s, %s, %s)", (job_title, company_name, location))
    conn.commit()
    
conn.close()

# Close the browser
driver.quit()

print("Data stored in the database successfully.")


OperationalError: connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "mydatabase" does not exist
